In [ ]:
import random as rd
import itertools

def QuickGamePlays(Strat1,Strat2,LRounds) -> tuple[float,float,int,int]:
    P1State, P2State = [None]*LRounds, [None]*LRounds
    for i in range(LRounds):
        P1Statecopy, P2Statecopy = P1State.copy(), P2State.copy()
        P1State[i] = Strat1(i,P1Statecopy,P2Statecopy)[i]
        P2State[i] = Strat2(i,P2Statecopy,P1Statecopy)[i]
    return P1State, P2State

def softMAJ(nthround,P1State,P2State):
    P1State[nthround] = "c" if 2*P2State.count("d") <= nthround else "d"
    return P1State

def ForgivingSoftMAJ(nthround,P1State,P2State):
    if nthround == 27:
        nc2 = 2*P2State.count("c")
        if nc2 >= P1State.count("c") and nc2 >= nthround - 1:
            P1State[nthround] = "c"
            return P1State

    P1State[nthround] = softMAJ(nthround,P1State,P2State)[nthround]
    return P1State

def ScoreTeam(SuperSecretCodes='random'):
    lenCode = 8
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    PaceMakerMoves = SuperSecretCodes[1] + ['c']*(200 - lenCode)
    RunnerMoves = QuickGamePlays(ForgivingSoftMAJ, (lambda a, b, c : PaceMakerMoves), lenCode)[0] + SuperSecretCodes[0] + ['d']*(200 - 2*lenCode)

    def ScoreRunner(nthround,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:max(lenCode, nthround)] == PaceMakerMoves[:max(lenCode, nthround)]:
            P1State[nthround] = RunnerMoves[nthround]
            return P1State
        
        if 199 - nthround < 1:
            P1State[nthround] = "d"
            return P1State

        # Otherwise, play RATIO
        P1State[nthround] = ForgivingSoftMAJ(nthround,P1State,P2State)[nthround]
        return P1State

    def ScorePaceMaker(nthround,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:nthround] == RunnerMoves[:nthround]:
            P1State[nthround] = PaceMakerMoves[nthround]
            return P1State
        
        # Otherwise, play ALLD
        P1State[nthround]="d"
        return P1State
    
    return ScoreRunner, ScorePaceMaker